# Transfer learning using Universal Sentence Encoder

## Import packages

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

Using TensorFlow backend.


## Downloading the model

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Elephant
Embedding size: 512
Embedding: [0.04498475790023804, -0.057433921843767166, 0.002211492508649826, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.055680178105831146, -0.00960793811827898, 0.00624628784134984, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.03874940052628517, 0.0765201598405838, -0.0007945735123939812, ...]



In [0]:
#size of embedding
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size


512

## Download data

In [0]:
!wget https://raw.githubusercontent.com/Tony607/Keras-Text-Transfer-Learning/master/train_5500.txt
!wget https://raw.githubusercontent.com/Tony607/Keras-Text-Transfer-Learning/master/test_data.txt

--2019-11-11 11:24:03--  https://raw.githubusercontent.com/Tony607/Keras-Text-Transfer-Learning/master/train_5500.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335860 (328K) [text/plain]
Saving to: ‘train_5500.txt’

train_5500.txt      100%[===================>] 327.99K  --.-KB/s    in 0.05s   

2019-11-11 11:24:03 (6.26 MB/s) - ‘train_5500.txt’ saved [335860/335860]

--2019-11-11 11:24:05--  https://raw.githubusercontent.com/Tony607/Keras-Text-Transfer-Learning/master/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2335

## Decscription of Data

The dataset we use is the TREC Question Classification dataset, There are entirely 5452 training and 500 test samples, that is 5452 + 500 questions each categorized into one of the six labels.

- ABBR - 'abbreviation': expression abbreviated, etc.
- DESC - 'description and abstract concepts': manner of an action, description of sth. etc.
- ENTY - 'entities': animals, colors, events, food, etc.
- HUM - 'human beings': a group or organization of persons, an individual, etc.
- LOC - 'locations': cities, countries, etc.
- NUM - 'numeric values': postcodes, dates, speed,temperature, etc

In [0]:
#Extract lines from .txt and convert to dataframe

def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(":")[0]
        text = ' '.join(lines[i].split(' ')[1:])
        text = re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df



In [0]:
#Assign train data
df_train = get_dataframe('train_5500.txt')
print(df_train.head())
df_test = get_dataframe('test_data.txt')
print(df_test.head())

  label                                               text
0  DESC  How did serfdom develop in and then leave Russ...
1  ENTY   What films featured the character Popeye Doyle ?
2  DESC  How can I find a list of celebrities ' real na...
3  ENTY  What fowl grabs the spotlight after the Chines...
4  ABBR                    What is the full form of .com ?
  label                                      text
0   NUM      How far is it from Denver to Aspen ?
1   LOC  What county is Modesto , California in ?
2   HUM                         Who was Galileo ?
3  DESC                         What is an atom ?
4   NUM          When did Hawaii become a state ?


In [0]:
#Number of categories in dataset
category_counts = len(df_train.label.cat.categories)
category_counts

6

In [0]:
#Convert input into strings  
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

## Model Building

In [0]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 132,870
Trainable params: 132,870
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Convert train text to  an array and labels as get_dummies
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [0]:
df_test = get_dataframe('test_data.txt')

In [0]:
#Convert test text to  an array and labels as get_dummies
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

## Train the model

In [0]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=32)
  model.save_weights('./model.h5')

Train on 5452 samples, validate on 500 samples
Epoch 1/10
5452/5452 [==============================] - 69s 13ms/step - loss: 0.7248 - acc: 0.8118 - val_loss: 0.3012 - val_acc: 0.9160
Epoch 2/10
5452/5452 [==============================] - 67s 12ms/step - loss: 0.3228 - acc: 0.8958 - val_loss: 0.2413 - val_acc: 0.9220
Epoch 3/10
5452/5452 [==============================] - 66s 12ms/step - loss: 0.2801 - acc: 0.9017 - val_loss: 0.2212 - val_acc: 0.9340
Epoch 4/10
5452/5452 [==============================] - 66s 12ms/step - loss: 0.2600 - acc: 0.9103 - val_loss: 0.2181 - val_acc: 0.9300
Epoch 5/10
5452/5452 [==============================] - 66s 12ms/step - loss: 0.2461 - acc: 0.9145 - val_loss: 0.2102 - val_acc: 0.9380
Epoch 6/10
5452/5452 [==============================] - 66s 12ms/step - loss: 0.2330 - acc: 0.9228 - val_loss: 0.2281 - val_acc: 0.9260
Epoch 7/10
5452/5452 [==============================] - 67s 12ms/step - loss: 0.2211 - acc: 0.9263 - val_loss: 0.2017 - val_acc: 0.9360
E

## Test the model

In [0]:
new_text = ["In what year did the titanic sink ?", 
            "What is the highest peak in California ?", 
            "Who invented the light bulb ?"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)

categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

['NUM', 'LOC', 'HUM']
